# RIDERneuro overview

In [ ]:
import sys
sys.path.insert(0, '../src')

import original.T1_mapping.MJT_EdinburghUK.t1 as edinburgh
import original.T1_mapping.ST_SydneyAus.VFAT1mapping as sydney
import original.T1_mapping.McGill.vfa as mcgill
import matplotlib.pyplot as plt
import os
import json
import re
import urllib.request
import nibabel as nib
import mat73
import numpy as np
from numpy.linalg import norm

plt.style.use('https://gist.github.com/notZaki/8bfb049230e307f4432fd68d24c603a1/raw/c0baa2a1c55afdf1764b26ee2ebeb1cbf26d8d98/pltstyle')

In [ ]:
def show_maps(fittedmaps, title = ""):
    fittedM0, fittedT1 = fittedmaps
    
    climM0 = np.nanquantile(fittedM0, [0.1, 0.98])
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6))
    imM0 = ax1.imshow(fittedM0, clim = climM0)
    ax1.grid(False)
    plt.sca(ax1)
    plt.title("M0")
    fig.colorbar(imM0, ax=ax1, orientation='vertical', shrink = 0.5)
    
    climT1 = np.nanquantile(fittedT1[fittedT1>0], [0.1, 0.98])
    imT1 = ax2.imshow(fittedT1, clim = climT1)
    ax2.grid(False)
    plt.sca(ax2)
    plt.title("T1")
    fig.colorbar(imT1, ax=ax2, orientation='vertical', shrink = 0.5)
    
    fig.suptitle(title)
    fig.tight_layout()
    return

def vfa_fit(_signal, _fa, _tr, author, fittype = "linear", mask = None):
    # Signal: numpy array, last dimension must be flip angle
    # fa: flip angles, in radian
    # tr: repetition time, in ms
    
    spatialdims = _signal.shape[:-1]
    signal = _signal.reshape(-1, _signal.shape[-1])
    if mask != None:
        signal = signal[mask[:] > 0, :]
        
    if author == "edinburgh":
        fa = _fa
        tr = _tr
        if fittype == "nonlinear":
            _M0, _T1 = edinburgh.fit_vfa_nonlinear(signal, fa, tr)
        else: # linear
            _M0, _T1 = edinburgh.fit_vfa_linear(signal, fa, tr)
    elif author == "sydney":
        fa = np.rad2deg(_fa)
        tr = _tr
        numvox = np.prod(spatialdims)
        _M0 = np.zeros(numvox)
        _T1 = np.zeros(numvox)
        for idx in range(numvox):
            _M0[idx], _T1[idx] = sydney.VFAT1mapping(fa, signal[idx, :], tr, method = fittype)
    elif author == "mcgill":
        fa = _fa
        tr = _tr
        if fittype == "nonlinear":
            _M0, _T1 = mcgill.novifast(signal, fa, tr, initialvalues=[10000, 3000])
        elif fittype == "nonlinear_noniterative":
            _M0, _T1 = mcgill.novifast(signal, fa, tr, doiterative = False)
        else: # linear
            _M0, _T1 = mcgill.despot(signal, fa, tr)
    else:
        print("ERROR: Unexpected author")
        return
    
    if mask != None:
        M0 = np.zeros(spatialdims)
        T1 = np.zeros(spatialdims)
        M0[mask[:] > 0] = _M0
        T1[mask[:] > 0] = _T1
    else:
        M0 = _M0.reshape(spatialdims)
        T1 = _T1.reshape(spatialdims)
    return (M0, T1)

def remove_leading_slash(pathstring):
    if pathstring[0] == "/":
        return pathstring[1:]
    else:
        return pathstring


def make_folder_for(file):
    folder = os.path.dirname(file)
    if not os.path.isdir(folder):
        os.makedirs(folder)
    return


def get_manifest(manifestfile, manifesturl = "https://osf.io/yhbsq/download"):
    if not os.path.isfile(manifestfile):
        urllib.request.urlretrieve(manifesturl, manifestfile)
    with open(manifestfile) as iostream:
        manifest = json.load(iostream)
    return manifest


def get_rider(subjectid, session, filetype, datafolder = "./data"):
    # subjectid in ["01" to "19"]
    # session in [1 or 2]
    # filetype in [VFA, GRE, perf, mask]

    # Download/get manifest
    if not os.path.isdir(datafolder): 
        os.makedirs(datafolder)
    manifestfile = os.path.join(datafolder, "RIDERmanifest.json")
    manifest = get_manifest(manifestfile)

    # Find key in manifest
    matchedkeys = []
    for key in manifest.keys():
        if (f"sub-{subjectid}" in key) and (f"ses-{session}" in key) and (filetype in key):
            matchedkeys.append(key)

    # Download/get files
    files = []
    for key in matchedkeys:
        fileurl = manifest[key]
        filepath = os.path.join(datafolder, remove_leading_slash(key))
        make_folder_for(filepath)
        if not os.path.isfile(filepath):
            urllib.request.urlretrieve(fileurl, filepath)
        files.append(filepath)
        
    return files

def get_flip_angle(filename):
    return int(re.search("flip(.*?)_VFA", filename).group(1)) * np.pi/180

def get_nifti_data(filename):
    return nib.load(filename).get_fdata()

def get_tr(filename):
    return nib.load(filename).header.get_zooms()[-1]

## Down/load data

The `get_rider` function can download data from the [RIDERneuro collection](https://wiki.cancerimagingarchive.net/display/Public/RIDER+NEURO+MRI) in the NIfTI format.
The input arguments are:

- `subjectid` can be between "01" to "19" for the 19 subjects in the collection
- `session` can either be "1" or "2" and corresponds to the two visits
- `filetype` will determine which data will be downloaded. The options include:
    - `"VFA.nii.gz"` for variable flip angle series (multiple files)
    - `"perf.nii.gz"` for dynamic contrast enhanced series
    - `"lesion_mask.nii.gz"` for a manually drawn tumour mask, in DCE-space
    - `"aif_mask.nii.gz"` for a semi-automatic mask for arterial/vein voxels, in DCE-space
    - `"muscle_mask.nii.gz"` for a manually drawn mask of the (temporalis?) muscle, in DCE-space
    - `"CSF_probseg.nii.gz"` for a probabilistic segmentation of cerebrospinal fluid regions, in VFA-space
    - `"WM_probseg.nii.gz"` for a probabilistic segmentation of white matter, in VFA-space
    - `"GM_probseg.nii.gz"` for a probabilistic segmentation of grey matter, in VFA-space
    
Calling `get_rider` will download the data to `./data/RIDERneuro/` and it will return the path to these files. 
If the files already exist, then nothing new will be downloaded and the path to the existing files will be returned.

The NIfTI files can be loaded using `nibabel.load`. For convenience, there is a `get_nifti_data` function defined in this notebook which will pull out the voxel data from the NIfTI file.

In [ ]:
subjectid = "02"
session = "1"

# VFA data
vfafiles = get_rider(subjectid = subjectid, session = session, filetype = "VFA.nii.gz")
vfasignal = np.stack([get_nifti_data(file) for file in vfafiles], axis = -1)
flipangles = np.array([get_flip_angle(file) for file in vfafiles])
tr = get_tr(vfafiles[0])

# DCE data
dcefile = get_rider(subjectid = subjectid, session = session, filetype = "dce_perf.nii.gz")[0]
dcesignal = get_nifti_data(dcefile)

# Mask
lesionmaskfile = get_rider(subjectid = subjectid, session = session, filetype = "lesion_mask.nii.gz")[0]
lesionmask = get_nifti_data(lesionmaskfile) 

aifmaskfile = get_rider(subjectid = subjectid, session = session, filetype = "aif_mask.nii.gz")[0]
musclemaskfile = get_rider(subjectid = subjectid, session = session, filetype = "muscle_mask.nii.gz")[0]

## Viewing the DCE and VFA series

Let's look at some of the downloaded files from the previous step.
For simplicity, only a single slice will be shown.

In [ ]:
chosenslice = 5
vfasignal = vfasignal[:,:,chosenslice,:]
dcesignal = dcesignal[:,:,chosenslice,:]
lesionmask = lesionmask[:,:,chosenslice]

The VFA data consists of the same volume acquired using 6 different flip angles.

In [ ]:
fig, _axes = plt.subplots(2, 3, figsize=(16,10))
for (idx, ax) in enumerate(fig.axes):
    ax.imshow(vfasignal[:,:,idx], clim = (20, 250))
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    ax.grid(False)
    plt.sca(ax)
    fa = idx*5 + 5
    plt.title(f"{fa} degree flip angle")

The magnetization (M0) and T1 values can be estimated from the VFA data.

In [ ]:
%time mcgill_lls = vfa_fit(vfasignal, flipangles, tr, author = "mcgill", fittype = "linear")
show_maps(mcgill_lls, title = "T1 mapping with linear least squares");

The estimated T1 values can then be used to convert the DCE-MRI signal into tracer concentration (not yet implemented).

The DCE-MRI series is shown below, along with the mean signal in each of the masked regions.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,6))
ax1.imshow(dcesignal[:,:,0], clim = (20, 125))
ax1.grid(False)
ax1.axes.xaxis.set_visible(False)
ax1.axes.yaxis.set_visible(False)
plt.sca(ax1)
plt.title("Pre-contrast")

ax2.imshow(dcesignal[:,:,-1], clim = (20, 125))
ax2.grid(False)
ax2.axes.xaxis.set_visible(False)
ax2.axes.yaxis.set_visible(False)
plt.sca(ax2)
plt.title("Post-contrast")
fig.tight_layout()

ax3.imshow(dcesignal[:,:,-1], clim = (20, 125))
ax3.contour(lesionmask, alpha = 0.5, colors='k')
ax3.grid(False)
ax3.axes.xaxis.set_visible(False)
ax3.axes.yaxis.set_visible(False)
plt.sca(ax3)
plt.title("Post-contrast with lesion contour")
fig.tight_layout()

In [ ]:
from nilearn.masking import apply_mask

lesioncurves = apply_mask(dcefile, lesionmaskfile)
aifcurves = apply_mask(dcefile, aifmaskfile)
musclecurves = apply_mask(dcefile, musclemaskfile)

fig, ax = plt.subplots(1, 1, figsize=(16,6))
ax.plot(lesioncurves.mean(axis = 1), label = "Lesion")
ax.plot(aifcurves.mean(axis = 1), label = "Blood vessel")
ax.plot(musclecurves.mean(axis = 1), label = "Muscle")
plt.title("Mean signal-time curve in different manually defined regions")
ax.legend();